# 설치구간



In [ ]:
import gc
gc.collect()

77

In [ ]:
import torch 
torch.cuda.empty_cache()

In [ ]:
!pip install wandb albumentations==1.3.0
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1b805e839d72d3d83d88702893c48034c1ba292d8fc67ee4cea51216a781f8ed
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Un

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
print(f'version A:{A.__version__}')
print(f'version cv2:{cv2.__version__}')
print(f'version torch :{torch.__version__}')

version A:1.3.0
version cv2:4.6.0
version torch :1.13.1+cu116


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from zipfile import ZipFile
import shutil

# 1BSlUkFetHU9BotXiK_pSRr6-oEk8vntF
# 1LP8ni_vGqDTyyTKJWaXtcgzY1Rh3-Otu
# 1iEOYgiwFLozIkZzLjKkmWtd9wMo_3JLa
!gdown "1BSlUkFetHU9BotXiK_pSRr6-oEk8vntF&confirm=t"
!gdown "1LP8ni_vGqDTyyTKJWaXtcgzY1Rh3-Otu&confirm=t"
!gdown "1iEOYgiwFLozIkZzLjKkmWtd9wMo_3JLa&confirm=t"

dir = '/content/train_sub_bgr'
ZipFile("open.zip").extractall('/content')
ZipFile("Gen_Images.zip").extractall('/content')
ZipFile("new_train_bgr.zip").extractall('/content')


Downloading...
From: https://drive.google.com/uc?id=1BSlUkFetHU9BotXiK_pSRr6-oEk8vntF&confirm=t
To: /content/open.zip
100% 754M/754M [00:07<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LP8ni_vGqDTyyTKJWaXtcgzY1Rh3-Otu&confirm=t
To: /content/Gen_Images.zip
100% 602M/602M [00:07<00:00, 84.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iEOYgiwFLozIkZzLjKkmWtd9wMo_3JLa&confirm=t
To: /content/new_train_bgr.zip
100% 1.99G/1.99G [00:23<00:00, 83.9MB/s]


In [ ]:
# try:
#     %cd "/content/drive/MyDrive/Colab Notebooks/"    
#     os.mkdir('block_dacon')
    
# except:
#     pass
# %cd "/content/drive/MyDrive/Colab Notebooks/block_dacon"
# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/block_dacon.zip"

#CFG

In [ ]:
accum = 16
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE': 16,
    'SEED':41,
    'NUM_WORKERS': 4
    }

file_name = "Coco_20230131_Resnet_bs-16_epoch-20_lr-3e-4_sig"

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# Wandb

In [ ]:
import wandb

wandb.login(key="c57ca8887aaf8537507fac6b7aa99f789d9d2e38", relogin=True)
#c57ca8887aaf8537507fac6b7aa99f789d9d2e38

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="test-project", entity="dcv_block-")
wandb.run.name = file_name

wandb.config.update(CFG)

wandb: Currently logged in as: h1lee (dcv_block-). Use `wandb login --relogin` to force relogin


# Model Download

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
!pip install validators torchvision 
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=6a7068fc87b1b0d6aa15609f7ec78392b09cd160e80c6544b9fd5369d8d325fa
  Stored in directory: /root/.cache/pip/wheels/19/09/72/3eb74d236bb48bd0f3c6c3c83e4e0c5bbfcbcad7c6c3539db8
Successfully built validators


Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
Downloading: "https://api.ngc.nvidia.com/v2/models/nvidia/resnet50_pyt_amp/versions/20.06.0/files/nvidia_resnet50_200821.pth.tar" to /root/.cache/torch/hub/checkpoints/nvidia_resnet50_200821.pth.tar


  0%|          | 0.00/97.7M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


# Data Load

In [ ]:
df = pd.read_csv('./train.csv')
num = int(df['id'].iloc[-1][6:11])
for i in range(num+1): 
  new = df.iloc[i].copy()
  new['id'] = f"TRAIN_{str(i+num+1).zfill(5)}"
  new['img_path'] = f"./Gen_Images/TRAIN_{str(i+num+1).zfill(5)}.jpg"
  df = df.append(new, ignore_index =True)
  if i + num + 1 == 65987:
    break

for i in range(num+1): 
  new = df.iloc[i].copy()
  new['id'] = f"TRAIN_{str(i+(num*2)+2).zfill(5)}"
  new['img_path'] = f"./new_train_bgr/TRAIN_{str(i+num*2+2).zfill(5)}.jpg"
  df = df.append(new, ignore_index =True)
  if i + num*2 + 2 == 98981:
    break

df = df.sample(frac=1)
train_len = int(len(df) * 0.8)

train = df[:train_len]
val = df[train_len:]

In [ ]:
def get_labels(df):
    return df.iloc[:,2:].values

train_labels = get_labels(train)
val_labels = get_labels(val)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

# Model define

In [ ]:
class Resnet50(nn.Module):
    def __init__(self, num_classes =10):
        super(Resnet50, self).__init__()
        self.backbone = resnet50
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [ ]:
model = Resnet50().to(device)
summary(model, input_size = (3, 224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
import datetime 
from accelerate import Accelerator
accelerator = Accelerator()

def trainer(model, optimizer, train_loader, val_loader, scheduler, device):
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    t_start = datetime.datetime.now()
    print(f'학습시작: {t_start}')
    for epoch in range(1, CFG['EPOCHS']+1):
        t_epoch_start = datetime.datetime.now()
        print(f'{epoch}번 epoch 시작: {t_epoch_start}')
        model.train()
        train_loss = []
        for i , data in tqdm((enumerate(tqdm(train_loader),0))):
            imgs , labels = data
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            output = model(imgs)
            loss = criterion(output, labels)
            loss = loss
            loss.backward()
            if (i+1) % accum ==0:
                optimizer.step()
                optimizer.zero_grad()
            train_loss.append(loss.item())
        t_epoch_end = datetime.datetime.now()
        print(f'{epoch}번 epoch 끝: {t_epoch_end}\n {epoch}번 소요시간: {t_epoch_end-t_epoch_start}')

        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        wandb.log({"train_loss": _train_loss, "val_loss":_val_loss})
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    t_end = datetime.datetime.now()
    t_gap_total= t_end -t_start 
    print(f'학습종료: {t_end}\n총 소요시간:{t_gap_total}')
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
   
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
          
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

# Run Train

In [ ]:
model = Resnet50()
model.eval().to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
wandb.watch(model, log_freq =100)
infer_model = trainer(model, optimizer, train_loader, val_loader,scheduler =scheduler, device =device)
#scheduler down

학습시작: 2023-01-31 02:07:37.838345
1번 epoch 시작: 2023-01-31 02:07:37.839777


  0%|          | 0/4950 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1번 epoch 끝: 2023-01-31 02:23:13.227824
 1번 소요시간: 0:15:35.388047


  0%|          | 0/1238 [00:00<?, ?it/s]

...

  0%|          | 0/4950 [00:00<?, ?it/s]

0it [00:00, ?it/s]

20번 epoch 끝: 2023-01-31 07:46:43.340579
 20번 소요시간: 0:15:43.819574


  0%|          | 0/1238 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00014] Val Loss : [0.00892] Val ACC : [0.99835]
학습종료: 2023-01-31 07:48:04.621135
총 소요시간:5:40:26.782790


In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(model, test_loader, device)

  0%|          | 0/92 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv(f'./sample_submission.csv')
submit.iloc[:,1:] = preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,0,0,0,1,1,0,0,0,0
2,TEST_00002,1,1,0,1,1,1,0,1,0,1
3,TEST_00003,0,1,0,0,1,1,0,1,1,1
4,TEST_00004,0,1,0,0,1,0,0,0,0,1


In [ ]:
submit.to_csv(f'./drive/MyDrive/Colab Notebooks/{file_name}_submit.csv', index=False)

FileNotFoundError: ignored

In [ ]:
torch.save(model, f'./{file_name}')